In [24]:
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 

# reading and preprocessing images
datagen = ImageDataGenerator(validation_split=0.3, rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)

batch_size = 256
img_size = 28
classes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

train_it = datagen.flow_from_directory('/Users/ayoungkim/Desktop/Pattern Recognition and Machine Learning/Assignement1/train/', batch_size=batch_size, 
                                       target_size=(img_size, img_size), subset='training', 
                                       class_mode='sparse', color_mode='grayscale', classes=classes)
val_it = datagen.flow_from_directory('/Users/ayoungkim/Desktop/Pattern Recognition and Machine Learning/Assignement1/train/', batch_size=batch_size, target_size=(img_size, img_size), 
                                    subset='validation', class_mode='sparse', color_mode='grayscale', 
                                     classes=classes)
test_it = datagen_test.flow_from_directory('/Users/ayoungkim/Desktop/Pattern Recognition and Machine Learning/Assignement1/test/', class_mode=None, batch_size=batch_size, 
                                      target_size=(img_size, img_size), shuffle=False, color_mode='grayscale')


Found 42000 images belonging to 10 classes.
Found 18000 images belonging to 10 classes.
Found 10000 images belonging to 1 classes.


In [2]:
# model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Input
from tensorflow.keras.models import Sequential

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

total_classes = 10
conv_layers = 3

model = Sequential()
model.add(Input(shape=(img_size,img_size, 1)))
for i in range(conv_layers):
    model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(total_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [3]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0

In [4]:
model.fit(
        train_it,
        epochs=60,
        validation_data=val_it)

Epoch 1/60
165/165 [==============================] - 70s 424ms/step - loss: 1.9179 - accuracy: 0.3018 - val_loss: 1.4539 - val_accuracy: 0.4641
Epoch 2/60
165/165 [==============================] - 63s 382ms/step - loss: 1.2696 - accuracy: 0.5292 - val_loss: 0.8425 - val_accuracy: 0.7286
Epoch 3/60
165/165 [==============================] - 60s 361ms/step - loss: 0.9531 - accuracy: 0.6461 - val_loss: 0.7509 - val_accuracy: 0.7291
Epoch 4/60
165/165 [==============================] - 60s 363ms/step - loss: 0.7511 - accuracy: 0.7257 - val_loss: 0.5727 - val_accuracy: 0.7911
Epoch 5/60
165/165 [==============================] - 61s 367ms/step - loss: 0.5933 - accuracy: 0.7844 - val_loss: 0.4023 - val_accuracy: 0.8708
Epoch 6/60
165/165 [==============================] - 61s 369ms/step - loss: 0.4753 - accuracy: 0.8286 - val_loss: 0.3427 - val_accuracy: 0.8806
Epoch 7/60
165/165 [==============================] - 58s 353ms/step - loss: 0.3851 - accuracy: 0.8620 - val_loss: 0.3860 - val_ac

165/165 [==============================] - 67s 404ms/step - loss: 0.0069 - accuracy: 0.9977 - val_loss: 0.0560 - val_accuracy: 0.9877
Epoch 58/60
165/165 [==============================] - 77s 470ms/step - loss: 0.0075 - accuracy: 0.9977 - val_loss: 0.0441 - val_accuracy: 0.9901
Epoch 59/60
165/165 [==============================] - 61s 370ms/step - loss: 0.0056 - accuracy: 0.9982 - val_loss: 0.0415 - val_accuracy: 0.9907
Epoch 60/60
165/165 [==============================] - 59s 357ms/step - loss: 0.0063 - accuracy: 0.9977 - val_loss: 0.0527 - val_accuracy: 0.9888


In [25]:
import numpy as np
import pathlib
import csv


# predict 
def prediction_file(images, ids, output_path):
    prediction_logits = model.predict(images)
    prediction = np.argmax(prediction_logits, axis=1) + 1 # classes: 1-10
    
    rows = [[id_file, yhat] for id_file, yhat in zip(ids, list(prediction))]
    
    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Id', 'Category'])
        writer.writerows(rows)
    

test_paths = sorted([path.stem for path in pathlib.Path('/Users/ayoungkim/Desktop/Pattern Recognition and Machine Learning/Assignement1/test/').glob('*test/*.jpg')])
# Make csv file first and copy the path to output_path
prediction_file(test_it, test_paths, '/Users/ayoungkim/Desktop/increased_epoch.csv')

for i in range(10):
    example_paths = sorted([path.stem for path in pathlib.Path('/Users/ayoungkim/Desktop/Pattern Recognition and Machine Learning/Assignement1/train/').glob('*'+str(i+1)+'/*.jpg')])
    example_it = datagen_test.flow_from_directory('/Users/ayoungkim/Desktop/Pattern Recognition and Machine Learning/Assignement1/train/', batch_size=batch_size, 
                                              target_size=(img_size, img_size),
                                          class_mode='sparse', color_mode='grayscale', 
                                          shuffle=False, classes=[str(i+1)])
    prediction_logits = model.predict(example_it)
    prediction = np.argmax(prediction_logits, axis=1) + 1 # classes: 1-10
    print(str(i+1) + ': ' + str(np.count_nonzero(prediction != (i+1))))
    
    # prediction_file(example_it, example_paths, 'exe_'+str(i+1)+'.csv')

Found 6000 images belonging to 1 classes.
1: 45
Found 6000 images belonging to 1 classes.
2: 4
Found 6000 images belonging to 1 classes.
3: 9
Found 6000 images belonging to 1 classes.
4: 40
Found 6000 images belonging to 1 classes.
5: 17
Found 6000 images belonging to 1 classes.
6: 27
Found 6000 images belonging to 1 classes.
7: 29
Found 6000 images belonging to 1 classes.
8: 3
Found 6000 images belonging to 1 classes.
9: 30
Found 6000 images belonging to 1 classes.
10: 4
